In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

REQUEST_URL = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13110&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1"
MAX_ITEMS = 1000
results = []
page_number = 1
item_count = 0

while item_count < MAX_ITEMS:
    res = f"{REQUEST_URL}&page={page_number}"
    response = requests.get(res)
    if response.status_code != 200:
        break

    soup = BeautifulSoup(response.text, "html.parser")
    cassette_items = soup.find_all(class_="cassetteitem")

    for item in cassette_items:
        # 各cassetteitemのタイトルなど共通情報を抽出
        title = item.find(class_="cassetteitem_content-title").text.strip()
        address = item.find(class_="cassetteitem_detail-col1").text.strip()

        # クラス名 'cassetteitem_detail-col3' を持つliタグを検索
        detail_col3 = soup.find('li', class_='cassetteitem_detail-col3')
        # 中のdiv要素を全て取得
        divs = detail_col3.find_all('div')
        tikunen = divs[0].text.strip() if len(divs) > 0 else None
        const = divs[1].text.strip() if len(divs) > 1 else None

        
        # 各cassetteitem内の複数のtbodyを取得
        tbodies = item.find_all('tbody')
        for tbody in tbodies:
            if item_count >= MAX_ITEMS:
                break
            # 各tbodyから間取り、料金、階数などの情報を抽出
            madori = tbody.find(class_="cassetteitem_madori").text.strip()
            menseki = tbody.find(class_="cassetteitem_menseki").text.strip()
            fee = tbody.find(class_="cassetteitem_other-emphasis ui-text--bold").text.strip()
            administration = tbody.find(class_="cassetteitem_price cassetteitem_price--administration").text.strip()
            deposit = tbody.find(class_="cassetteitem_price cassetteitem_price--deposit").text.strip()
            garatuity = tbody.find(class_="cassetteitem_price cassetteitem_price--gratuity").text.strip()
            floor = tbody.find_all('td')[2].text.strip() if tbody.find_all('td') else "不明"
            
            # 各物件の情報を辞書として追加
            bukken_info = {
                "名称": title,
                "アドレス": address,
                "築年数": tikunen,
                "構造": const,
                "間取り": madori,
                "面積": menseki,
                "家賃": fee,
                "管理費": administration,
                "敷金": deposit,
                "礼金": garatuity,
                "階数": floor

            }
            results.append(bukken_info)
            item_count += 1

    page_number += 1
    
    next_button = soup.select('p.pagination-parts > a')
    if not next_button or "次へ" not in next_button[-1].text or item_count >= MAX_ITEMS:
        break  # '次へ' ボタンがない、またはアイテム数がMAX_ITEMSに達したらループを終了

# データフレームを作成
df = pd.DataFrame(results)

#データの整形
# "万円"を削除して、文字列を数値に変換する関数定義
def convert_rent(value):
    if isinstance(value, str)and '万円' in value:
        return float(value.replace('万円', ''))
    else:
        return value
    
# "円"を削除して、文字列を数値に変換する関数定義
def convert_deposit(value):
    if isinstance(value, str)and '円' in value:
        return float(value.replace('円', ''))
    else:
        return value
    
df['家賃'] = df['家賃'].apply(convert_rent)
df['敷金'] = df['敷金'].apply(convert_rent)
df['礼金'] = df['礼金'].apply(convert_rent)
df['管理費'] = df['管理費'].apply(convert_deposit)

#築年数の変換
df['築年数'] = df['築年数'].replace("新築",0)
df['築年数'] = df['築年数'].replace(to_replace='築', value='', regex=True).replace(to_replace='年', value='', regex=True).astype(int)

# 重複する行を削除
unique_data = df.drop_duplicates(subset=['名称', 'アドレス', '階数', '家賃', '敷金', '礼金', '間取り'])

# CSVに出力
csv_file_path = 'unique_output.csv'
unique_data.to_csv(csv_file_path, index=False)

print(f"CSVファイルが出力されました: {csv_file_path}")

unique_data


Googleスプレッドシートを呼び出し、加工したデータを格納する

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

from dotenv import load_dotenv
load_dotenv()

import os

SPREADSHEET_KEY = os.getenv('SPREADSHEET_KEY')

In [ ]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
Auth = 'suumo-step3-1-33c7f663e7dd.json'


credentials = ServiceAccountCredentials.from_json_keyfile_name(Auth, scope)

gs = gspread.authorize(credentials)
worksheet = gs.open_by_key(SPREADSHEET_KEY).worksheet("シート1")

In [ ]:
from gspread_dataframe import set_with_dataframe

In [ ]:
workbook = gs.open_by_key(SPREADSHEET_KEY)

In [ ]:
workbook.add_worksheet(title="Tokyo", rows=1000, cols=20)

In [ ]:
set_with_dataframe(workbook.worksheet("Tokyo"), unique_data, include_index= True)